In [ ]:
from flask import Flask, redirect, url_for, session
from flask_oidc import OpenIDConnect
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

import pandas as pd
import io
import plotly.express as px

# Import Flask server from keycloak-auth.py
from keycloakAuth import flaskServer, oidc

# Initialize Dash app with Flask server
app = dash.Dash(__name__, server=flaskServer, url_base_pathname='/dashboard/')

# Import SimvaBrowser from simvaWidget.py
from simvaWidget import SimvaBrowser

In [ ]:
# Dash callback to handle login button click
@app.callback(
    Output('login-logout-button', 'children'),
    [Input('main', 'children')]
)
def login_logout_button(main):
    if oidc.user_loggedin:
        return "Logout"
    else:
        return "Login"
        
# Dash callback to handle login button click
@app.callback(
    Output('login-logout', 'children'),
    [Input('login-logout-button', 'n_clicks')]
)
def login_logout_button_click(n_clicks):
    if n_clicks > 0:
        return dcc.Location(pathname='/', id='login-logout-link')

# Dash callback to update connection status
@app.callback(
    Output('connection-status', 'children'),
    [Input('main', 'children')]
)
def update_connection_status(input_value):
    if oidc.user_loggedin:
        user_info = session.get('oidc_auth_profile', {})
        preferred_username = user_info.get('preferred_username')
        return f'Logged in as {preferred_username} - {session}'
    else:
        return 'Not logged in'

# Example of a protected route (not yet implemented)
@app.server.route('/dashboard/')
def render_dashboard():
    return app.index()

# Example of a logout button (not yet implemented)
@app.server.route('/logout')
def logout():
    return redirect(url_for('home'))  # Replace 'home' with your home route

In [ ]:
# Dash callback to handle login button click
@app.callback(
    Output('content', 'children'),
    [Input('main', 'children')]
)
def init_storage(main):
    if oidc.user_loggedin:
        # Initialize SimvaBrowser
        storage_url = "https://minio-api.simva-beta.e-ucm.es/"  # Provide your storage URL
        browser = SimvaBrowser(session, storage_url)

        appLayout = html.Div([
            html.H3(id='current-path', children=browser.current_path),
            html.Button('..', id='parent-directory', n_clicks=0),
            html.Div(id='folders-div'),
            html.Div(id='files-div'),
            html.Button('Run Analyse', id='run-analyse', style={'display': 'none'}),
            html.Div(id='output')
        ])

        @app.callback(
            [Output('current-path', 'children'),
            Output('folders-div', 'children'),
            Output('files-div', 'children'),
            Output('run-analyse', 'style')],
            [Input('parent-directory', 'n_clicks'),
            Input('folders-div', 'children'),
            Input('files-div', 'children')],
            [State('current-path', 'children')]
        )
        def update_browser(n_clicks, folders, files, current_path):
            ctx = dash.callback_context

            if not ctx.triggered:
                raise PreventUpdate

            button_id = ctx.triggered[0]['prop_id'].split('.')[0]

            if button_id == 'parent-directory':
                if len(browser.current_path) > len(browser.base_path):
                    browser.current_path = browser.current_path.rpartition(browser.delimiter)[0].rpartition(browser.delimiter)[0] + browser.delimiter
                else:
                    browser.current_path = browser.base_path
            else:
                browser.current_path = browser.current_path + button_id

            browser._update_files()

            folder_buttons = [html.Button(f, id=f, n_clicks=0) for f in browser.dirs]
            file_buttons = [html.Button(f, id=f, n_clicks=0, style={'background-color': 'green'}) for f in browser.files if f.endswith(browser.accept)]

            run_analyse_style = {'display': 'none'}
            if not browser._isdir(browser.current_path):
                run_analyse_style = {'display': 'block'}

            return browser.current_path, folder_buttons, file_buttons, run_analyse_style

        @app.callback(
            Output('output', 'children'),
            [Input('run-analyse', 'n_clicks')],
            [State('current-path', 'children')]
        )
        def run_analyse(n_clicks, current_path):
            if n_clicks > 0:
                content = browser.get_file_content(current_path)
                # Implement your analysis logic here
                # For example, assume the file content is a CSV and we want to plot some data
                df = pd.read_csv(io.StringIO(content.decode('utf-8')))
                fig = px.scatter(df, x=df.columns[0], y=df.columns[1])
                return dcc.Graph(figure=fig)
            else:
                raise PreventUpdate


In [ ]:
# Layout of the dashboard
app.layout = html.Div(id="main", children=[
    html.H1('Secure Dashboard'),
    html.Div(id='connection-status'),
    html.Div(id='content'), 
    html.Button(id='login-logout-button', n_clicks=0),
    html.Div(id='login-logout'),
])

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True, port=5000)